In [6]:
import os 
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI

try: 
    load_dotenv()
    os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
    print("Environment variables loaded successfully.")
except Exception as e:
    print(f"Error loading environment variables: {e}")
    
model = ChatOpenAI(model="gpt-4o-mini", temperature=0.8)

Environment variables loaded successfully.


In [23]:
from pypdf import PdfReader
from unstructured.partition.pdf import partition_pdf
from langchain.prompts import ChatPromptTemplate
from langchain.schema import StrOutputParser
from unstructured.partition.html import partition_html
from unstructured.partition.md import partition_md


class ExtractData:
    
    def extract_text_from_pdf(self, pdf_path):
        try:
            elements = partition_pdf(
            filename=pdf_path,                  # mandatory
            strategy="hi_res",                                     # mandatory to use ``hi_res`` strategy
            extract_images_in_pdf=True,                            # mandatory to set as ``True``
            extract_image_block_types=["Image", "Table"],          # optional
            extract_image_block_to_payload=True,    
            infer_table_structure=True,# optional
            extract_image_block_output_dir="images/",  # optional - only works when ``extract_image_block_to_payload=False``
            languages=["eng"],                           # optional
            )

            tables, images, texts = self.parse_elements(elements)
            tables_summary = self.create_summaries_of_tables(tables)
            texts_summary = self.create_summaries_of_texts(texts)
            images_summary = self.create_summaries_of_images(images)
            return tables_summary, images_summary, texts_summary

        except Exception as e:
            print(f"Error extracting text from PDF: {e}")
            return [], [], []
    
    def extract_text_from_html(self, html_path):
        try:
            # From filename
            elements = partition_html(filename=html_path)

            # From file object
            with open(html_path, "r") as f:
                elements = partition_html(file=f)

            # From text string
            with open(html_path, "r") as f:
                text = f.read()
            elements = partition_html(text=text)

            tables, images, texts = self.parse_elements(elements)
        
        except Exception as e:
            print(f"Error in Extraction of HTML : {str(e)}")
            return [], [], []
                
        
    
    def extract_text_from_markdown(self, md_path):
        try:
            # From filename
            elements = partition_md(filename=md_path)

            # # From file object
            with open(md_path, "r") as f:
                elements = partition_md(file=f)

            # # From text string
            with open(md_path, "r") as f:
                text = f.read()
            elements = partition_md(text=text)
            tables, images, texts = self.parse_elements(elements)
            return tables, images, texts
        except Exception as e:
            print(f"Error in Extraction of Markdown : {str(e)}")
            return [], [], []
    
    def parse_elements(self, elements):
        try:
            tables = []
            images = []
            texts = []

            # Save image and table elements
            for i, element in enumerate(elements):
                # print(f"{i}. Type: {element.category}")
                
                if element.category == "Image":
                    #print(element.metadata.image_base64)
                    images.append(element.metadata.image_base64)
                
                elif element.category == "Table":
                    #print("Table HTML:")
                    tables.append(element.metadata.text_as_html)
                else:
                    texts.append(element.text)
            
            return tables, images, texts
        
        except Exception as e:
            print(f"Error Parsing Elements: {e}")
            return [], [], []
        
    
    def create_summaries_of_tables(self, tables):
        # Prompt
        prompt_text = """You are an assistant tasked with summarizing tables. \
        Give a concise summary of the table. Table chunk: {element} """
        prompt = ChatPromptTemplate.from_template(prompt_text)

        # Summary chain
        summarize_chain = {"element": lambda x: x} | prompt | model | StrOutputParser()

        table_summaries = summarize_chain.batch(tables, {"max_concurrency": 5})
        return table_summaries

    def create_summaries_of_texts(self, texts):
        # Prompt
        prompt_text = """You are an assistant tasked with summarizing texts. \
        Give a concise summary of the text. Text chunk: {element} """
        prompt = ChatPromptTemplate.from_template(prompt_text)

        # Summary chain
        summarize_chain = {"element": lambda x: x} | prompt | model | StrOutputParser()

        text_summaries = summarize_chain.batch(texts, {"max_concurrency": 5})
        return text_summaries

    def create_summaries_of_images(self, images):
        prompt_template = """You are an assistant tasked with summarizing images for retrieval.
                Remember these images could potentially contain graphs, charts or 
                tables also.
                These summaries will be embedded and used to retrieve the raw image 
                for question answering.
                Give a detailed summary of the image that is well optimized for 
                retrieval.
                Do not add additional words like Summary: etc.
             """
        messages = [
            (
                "user",
                [
                    {"type": "text", "text": prompt_template},
                    {
                        "type": "image_url",
                        "image_url": {"url": "data:image/jpeg;base64,{image}"},
                    },
                ],
            )
        ]

        prompt = ChatPromptTemplate.from_messages(messages)

        chain = prompt | model | StrOutputParser()

        image_summaries = chain.batch(images)
        return image_summaries

In [24]:
tables, images, text = ExtractData().extract_text_from_pdf("data/aadhar.pdf")

In [ ]:
print(len(tables))

In [ ]:
print(len(images))

In [ ]:
print(text)

In [ ]:
print(ExtractData().create_summaries_of_tables(tables))

In [ ]:
gen_image_summaries = ExtractData().create_summaries_of_images(images)

In [ ]:
for i in gen_image_summaries:
    print(i)

In [ ]:
from unstructured.partition.html import partition_html

# From filename
elements = partition_html(filename="data/upi.html")

# From file object
with open("data/upi.html", "r") as f:
    elements = partition_html(file=f)

# From text string
with open("data/upi.html", "r") as f:
    text = f.read()
elements = partition_html(text=text)

html_tables = []
html_images = []
html_texts = []

# Save image and table elements
for i, element in enumerate(elements):
    # print(f"{i}. Type: {element.category}")
    
    if element.category == "Image":
        # print(element.metadata.to_dict())
        html_images.append(element.metadata.image_url)

    elif element.category == "Table":
        #print("Table HTML:")
        html_tables.append(element.metadata.text_as_html)
    else:
        html_texts.append(element.text)
        
print(html_tables)
print(html_texts)
print(html_images)



In [ ]:
from unstructured.partition.md import partition_md

# From filename
elements = partition_md(filename="data/README.md")

# From file object
with open("data/README.md", "r") as f:
    elements = partition_md(file=f)

# From text string
with open("data/README.md", "r") as f:
    text = f.read()
elements = partition_md(text=text)
print(elements)
#pip install "unstructured[md]"

In [ ]:
md_tables, md_images, md_texts = ExtractData().extract_text_from_markdown("data/README.md")

In [ ]:
print(md_tables)

In [ ]:
from unstructured.partition.md import partition_md

# Load and partition a Markdown file
elements = partition_md("data/README.md")

# Extract images
tb, im, tx = ExtractData().parse_elements(elements)

for img in images:
    print("Image metadata:", img.metadata.to_dict())
    print("Image text (alt text if available):", img.text)


In [ ]:
import base64
import requests

def urls_to_base64(urls):
    """Fetch images from a list of URLs and convert them to base64."""
    results = {}
    for url in urls:
        try:
            response = requests.get(url)
            response.raise_for_status()
            encoded = base64.b64encode(response.content).decode("utf-8")
            results[url] = encoded
        except Exception as e:
            results[url] = ""
    return results

# Example usage
image_urls = ['https://razorpay.com/blog-content/uploads/2020/06/Logo.svg', 
              'https://d6xcmfyh68wv8.cloudfront.net/blog-content/uploads/2024/02/upi-image-1024x536.webp', 
              'https://razorpay.com/blog-content/uploads/2021/01/upi-intent-payment.png', 
              'https://d6xcmfyh68wv8.cloudfront.net/blog-content/uploads/2021/01/unnamed.png', 
              'https://d6xcmfyh68wv8.cloudfront.net/blog-content/uploads/2021/01/UPI-usage.png', 
              'https://secure.gravatar.com/avatar/47772782b201521accc04b923d55ceec?s=82&d=mm&r=g', 
              'https://d6xcmfyh68wv8.cloudfront.net/blog-content/uploads/2025/08/hugg1-770x515.png',
              'https://d6xcmfyh68wv8.cloudfront.net/blog-content/uploads/2025/08/Slow-Moto-Tours-Blog-1-770x515.png']

base64_images = urls_to_base64(image_urls)

for url, encoded in base64_images.items():
    if encoded.startswith("Error"):
        print(f"⚠️ {url}: {encoded}")
    else:
        print(f"✅ {url} -> Base64 snippet: {encoded[:100]}...")


In [5]:
tables, images, text = ExtractData().extract_text_from_pdf("data/aadhar.pdf")

NameError: name 'ExtractData' is not defined

In [ ]:
# from llama_index.core import Document

# text_docs = [Document(page_content=t,metadata={"source": "pdf"},) for t in text]
# table_docs = [Document(page_content=t,metadata={"source": "pdf"},) for t in tables]
# image_docs = [Document(page_content=i,metadata={"source": "pdf"},) for i in images]



In [38]:
print(text_docs[0])

Doc ID: 4a8fd8fd-45dc-43cb-911e-fdced400f209
Text:


In [10]:
from llama_index.core.node_parser import (
    SentenceSplitter,
    SemanticSplitterNodeParser,
)
from llama_index.embeddings.openai import OpenAIEmbedding

import os

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
embed_model = OpenAIEmbedding()
splitter = SemanticSplitterNodeParser(
    buffer_size=1, breakpoint_percentile_threshold=95, embed_model=embed_model
)

# also baseline splitter
base_splitter = SentenceSplitter(chunk_size=512)
nodes = splitter.get_nodes_from_documents(documents)

KeyboardInterrupt: 

In [17]:
print(type(nodes[0]))

<class 'llama_index.core.schema.TextNode'>


In [33]:
print(images)

['Image features a close-up photograph of a man with a mustache and short black hair. He is wearing a white shirt and appears to have a neutral expression. The background is a plain light color, enhancing the focus on his face. The image is framed closely around the head and shoulders.', "The image features a headshot of a man with short black hair and glasses. He has a neutral expression and is wearing a light-colored shirt. The background is plain and light, enhancing the visibility of the subject's features. The image is a standard portrait style, focusing on the upper part of the individual, likely for professional or identification purposes.", 'The image depicts a network diagram for the NIC Aadhaar Authentication Services, showcasing various components and their interconnections. It includes multiple data centers and locations, specifically NIC Koramangala, CIDR UIDAI in Bangalore, DC Pune, DC Hyderabad, and DC Shastri Park. \n\nKey elements featured in the diagram are:\n\n- **NI

In [48]:
from uuid import uuid4

from langchain_core.documents import Document

text_docs = [Document(page_content=t,metadata={"source": "pdf"},) for t in text]
table_docs = [Document(page_content=t,metadata={"source": "pdf"},) for t in tables]
image_docs = [Document(page_content=i,metadata={"source": "pdf"},) for i in images]
text_uuids = [str(uuid4()) for _ in range(len(text))]
image_uuids = [str(uuid4()) for _ in range(len(images))]
table_uuids = [str(uuid4()) for _ in range(len(tables))]


In [47]:
# print(documents[0])
print(text_docs[0])

page_content='The text discusses the offerings related to electronic government (eGov) products and services, which are designed to enhance the efficiency and accessibility of government operations through digital solutions.' metadata={'source': 'pdf'}


In [49]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

In [50]:
import faiss
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS

index = faiss.IndexFlatL2(len(embeddings.embed_query("hello world")))

vector_store = FAISS(
    embedding_function=embeddings,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
)

In [51]:
vector_store.add_documents(documents=text_docs, ids=text_uuids)
vector_store.add_documents(documents=image_docs, ids=image_uuids)
vector_store.add_documents(documents=table_docs, ids=table_uuids)

['a031b5ad-e31e-482c-b490-6d947f8f7557']

In [59]:
from langchain.retrievers.multi_vector import MultiVectorRetriever
from langchain.storage import InMemoryStore
id_key = "doc_id"
store = InMemoryStore()
retriever = MultiVectorRetriever(
    vectorstore=vector_store,
    docstore=store,
    id_key=id_key,
)
retriever.vectorstore.add_documents(documents=text_docs, ids=text_uuids)
retriever.docstore.mset(list(zip(text_uuids, text)))

ValueError: Tried to add ids that already exist: {'09de9ea7-edc7-4983-8116-219d0054c7ee', '77838f34-84a0-47ad-8261-205c9f3bf0ac', '108e1c25-c401-4147-b0e7-e7f5fe6b03c9', '48ae3f4f-779e-4c85-b0b6-25f0420b4338', '5655d0a1-1935-478a-a1b1-a3b830352a2f', 'a83a072c-7219-4c7d-bd7b-2b4f69ba78d9', '28102c1a-9dfe-43cb-8c87-e82092bda8b0', 'a7a161f8-45c8-4f78-afd8-cb584f77f62b', '744d8215-1354-4d11-884c-8524858a33fe', '41e5e237-dde6-4a11-9ad8-51ae0c05df87', '5bb129de-c9ff-4ad7-ba7c-397dceca5ce7', '6a19be29-d48d-42bb-8707-a8d341db7115', '6c6b9b4c-2e48-4551-9d40-da48c8885ebb', 'ac21e15b-f7a9-4185-92dc-e83983d7093d', 'bc29817e-ec1a-4002-9c8b-8c598ae87977', '7e33d3bc-a655-4deb-b4f5-a3f1d26e1171', '3382d92c-e6ec-43be-aec7-1a9a69c9b4d4', '76b46dfc-234a-454e-9e17-d8011ccc379f', '3458104a-17fb-4186-bfca-4e3c8dfdb2e7', '078deb2f-8e8e-4d55-bfc9-ec2abd302c26', '8e3a26b2-fb5a-457f-a2b1-d3f3b78c7969', 'a5a36cd3-dfa8-412f-b5e2-a49a310456ec', '1d1f5b43-b512-43ef-97c0-9f445f2fc864', 'a2df19a9-25af-4061-a96d-0af3b

In [60]:
results = vector_store.similarity_search(
    "on boarding process",
    k=1
)
for res in results:
    print(f"* {res.page_content} [{res.metadata}]")

* The on-boarding process refers to the procedures and activities designed to integrate new employees into an organization. It typically includes orientation, training, and introduction to company culture and policies, aiming to ensure a smooth transition and enhance employee engagement and productivity. [{'source': 'pdf'}]


In [55]:
# for image_doc in image_docs:
#     print(image_doc.page_content)  # Print first 100 characters of each image document
image_docs[-1]

Document(metadata={'source': 'pdf'}, page_content='The image features a triangular structure with a list of various programs or initiatives organized vertically. Each program is represented with alternating colors—top sections in yellow and bottom sections in teal. The listed items from top to bottom are:\n\n1. Attendance\n2. SERVAM (NPR)\n3. ePDS (States)\n4. Scholarship\n5. Swachh Bharat Mission\n6. MSME\n7. Indian Army\n8. e-Panchayat (AP)\n9. CBSE, NET\n10. Pradhan Mantri Awas Yojna\n\nThe design emphasizes a hierarchical structure, possibly indicating a prioritization or categorization of these programs. The triangular format suggests an upward trend or importance of the listed initiatives.')

In [57]:
retriever = vector_store.as_retriever(search_type="mmr", search_kwargs={"k": 1})
retriever.invoke("ELIGIBILITY FOR GETTING AUTHENTICATION SERVICES")

[Document(id='734d59c4-4728-420c-b9e4-c6ec7ef71b2a', metadata={'source': 'pdf'}, page_content='The text discusses the criteria required for individuals or entities to qualify for authentication services.')]